In [1]:
%load_ext autoreload
%autoreload 2

C:\Users\balin\miniconda3\envs\deep_learning\lib\site-packages\ipykernel\parentpoller.py:113: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  warnings.warn("""Parent poll failed.  If the frontend dies,


## Data

In [2]:
# load WebBeteg db
import json

with open('./data/WebBeteg', 'rb') as f:
    database = json.load(f)['_default']

keys = list(database.keys())

webbeteg = ' '.join([database[k]['text'] for k in keys[:2000]])

In [4]:
# load shakespeare db

import tensorflow as tf
from src.data import Dataset

path_to_file = tf.keras.utils.get_file('shakespeare.txt', 
                                       'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
shakespeare = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [5]:
with open('./data/moliere_complete.txt', 'r') as f:
    moliere = ''.join(list(f))

In [6]:
dataset = Dataset(webbeteg)

## Model

In [7]:
from src.transformer_model import TransformerDecoder
from src.train import train

num_layers = 6
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

transformer = TransformerDecoder(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    target_vocab_size=len(dataset.vocab),
    maximum_position_encoding=100,
    rate=dropout_rate)

train(transformer,  dataset.dataset, 20)
transformer.summary()

Epoch 1/20
184/184 [==============================] - 18s 68ms/step - loss: 3.4996
Epoch 2/20
184/184 [==============================] - 13s 68ms/step - loss: 3.3253
Epoch 3/20
184/184 [==============================] - 13s 68ms/step - loss: 2.8761
Epoch 4/20
184/184 [==============================] - 13s 68ms/step - loss: 2.7125
Epoch 5/20
184/184 [==============================] - 14s 69ms/step - loss: 2.4524
Epoch 6/20
184/184 [==============================] - 14s 69ms/step - loss: 2.2077
Epoch 7/20
184/184 [==============================] - 14s 69ms/step - loss: 2.0523
Epoch 8/20
184/184 [==============================] - 14s 69ms/step - loss: 1.9467
Epoch 9/20
184/184 [==============================] - 13s 68ms/step - loss: 1.8603
Epoch 10/20
184/184 [==============================] - 13s 67ms/step - loss: 1.8045
Epoch 11/20
184/184 [==============================] - 13s 66ms/step - loss: 1.7625
Epoch 12/20
184/184 [==============================] - 13s 66ms/step - loss: 1.7296
E

In [8]:
from src.rnn_model import LSTMModel
from src.train import train

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 512

rnn_model = LSTMModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(dataset.vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

train(rnn_model, dataset.dataset, 20)
rnn_model.summary()

Epoch 1/20
184/184 [==============================] - 8s 26ms/step - loss: 3.4089
Epoch 2/20
184/184 [==============================] - 6s 26ms/step - loss: 2.5185
Epoch 3/20
184/184 [==============================] - 6s 27ms/step - loss: 2.2501
Epoch 4/20
184/184 [==============================] - 6s 27ms/step - loss: 2.0725
Epoch 5/20
184/184 [==============================] - 6s 26ms/step - loss: 1.9465
Epoch 6/20
184/184 [==============================] - 6s 27ms/step - loss: 1.8340
Epoch 7/20
184/184 [==============================] - 6s 26ms/step - loss: 1.7548
Epoch 8/20
184/184 [==============================] - 6s 27ms/step - loss: 1.6922
Epoch 9/20
184/184 [==============================] - 6s 26ms/step - loss: 1.6348
Epoch 10/20
184/184 [==============================] - 6s 26ms/step - loss: 1.5990
Epoch 11/20
184/184 [==============================] - 6s 27ms/step - loss: 1.5555
Epoch 12/20
184/184 [==============================] - 6s 27ms/step - loss: 1.5227
Epoch 13/20
1

## Text generation

In [54]:
result = []

for i in range(500):
    if i%4==0:
        start_char='A'
    elif i%4==1:
        start_char='T'
    elif i%4==2:
        start_char='E'
    elif i%4==3:
        start_char='F'
        print(i)
    result.append(rnn_generator.generate_text(start_char, 200))
with open('./result/lstm_shakespeare.pkl', 'wb+') as f:
    pkl.dump(result, f)

3
7
11
15
19
23
27
31
35
39
43
47
51
55
59
63
67
71
75
79
83
87
91
95
99
103
107
111
115
119
123
127
131
135
139
143
147
151
155
159
163
167
171
175
179
183
187
191
195
199
203
207
211
215
219
223
227
231
235
239
243
247
251
255
259
263
267
271
275
279
283
287
291
295
299
303
307
311
315
319
323
327
331
335
339
343
347
351
355
359
363
367
371
375
379
383
387
391
395
399
403
407
411
415
419
423
427
431
435
439
443
447
451
455
459
463
467
471
475
479
483
487
491
495
499


In [12]:
from src.rnn_model import RNNGenerator
import pickle as pkl

rnn_generator = RNNGenerator(rnn_model, dataset.chars_from_ids, dataset.ids_from_chars)
print(rnn_generator.generate_text('Doktor', 200))

Doktornő! azokban eltoltam, hogy genet azzal az első napig nem szoktam sem voltak nálam kisebb szaportot is, hogy szerette) Tisztelt Doktornő! Köszönöm segítségét és szeretnék a terhességi csikett és így eg


In [13]:
from src.transformer_model import TransformerGenerator

transformer_generator = TransformerGenerator(transformer, dataset.chars_from_ids, dataset.ids_from_chars)
print(transformer_generator.generate_text('Doktor', 200))

Doktornő/ 2 kivembészt: edett évett. Mi  lehet az endre a-V betegség, de napra ciklálom nagyon volt, felölg, hogy mit szét szorongathatok megegnjekci. 2 óra segítsen megijedve, s rossz.2 ma? A mikor ki erőt
